In [ ]:
#@title Imports
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.colheader_justify', 'center')
import numpy as np
import random
import time
import re

import scipy.stats
from scipy import stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

from inspect import signature

def someMethod(self, arg1, kwarg1=None):
    pass

nba_possession_data_seasons = np.arange(1974, 2024, 1)
play_by_play_data_seasons = np.arange(1997, 2024, 1)
nba_pre_possession_data_seasons = np.arange(1952, 1974, 1)
aba_possession_data_seasons = np.arange(1974, 1977, 1)
aba_pre_possession_data_seasons = np.arange(1968, 1974, 1)

all_nba_seasons = np.arange(1952, 2024, 1)
all_aba_seasons = np.arange(1968, 1977, 1)

# plot
import seaborn as sns
import ast
from matplotlib.lines import Line2D
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
sns.set_style('darkgrid')        # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
      #if i % 2 == 0:
      ax.text(point['x']+0.12, point['y']+.25, str(point['val']))
      #else:
      #ax.text(point['x']+0.30, point['y'], str(point['val']))
      #ax.text(point['x']+0.05, point['y'], str(point['val']))

def label_point_year(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
      #if i % 2 == 0:
      ax.text(point['x']+0.12, point['y'], int(point['val']))
      #else:
      #ax.text(point['x']+0.30, point['y'], str(point['val']))
      #ax.text(point['x']+0.05, point['y'], str(point['val']))

plot_colors_set_list = ["#FF0000", "#D3A6D6", "#916613", "#00A4A0", "#FF7F00", "#ABFFD2", "#610077", "#173E4C", "#00A54C", "#FF00A1", "#FFFE00", "#0B1ADD"]

In [ ]:
#@title Import Selenium
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
#@title Start Webdriver and VirtualDisplay
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(service=service, options=options)

#options = Options()
#options.add_argument("--headless")
#options.add_argument("--no-sandbox")
#options.headless = True

#wd = webdriver.Chrome("/usr/bin/chromedriver", options=options)

!pip install pyvirtualdisplay
!apt-get install xvfb
from pyvirtualdisplay import Display
display = Display(visible=0, size=(800, 800))
display.start()

**SERIES PERCENTILES**

In [ ]:
#@title Print PTS/TS+ percentiles for a players' series
series_data_52_22_df = pd.read_csv('/content/series_data_52_23.csv', encoding='utf8', index_col=False)
def printSeries_PTS_TS_Percentiles(p1, mp_min, pts_min):
  graph_data =  series_data_52_22_df.copy()
  graph_data = graph_data[(graph_data['PTS/G'] >= pts_min)]
  graph_data = graph_data[(graph_data['MP'] >= mp_min)]
  graph_data['Approx_Scoring_Val'] = 0

  for idx, row in graph_data.iterrows():
    pts = row['PTS/G']
    ts = row['TS+']
    mp = row['MP']
    pts_percentile = stats.percentileofscore(graph_data['PTS/G'], pts)
    ts_percentile = stats.percentileofscore(graph_data['TS+'], ts)
    approx_scoring_val = (0.5 * pts_percentile) + (0.5 * ts_percentile)
    graph_data.at[idx, 'Approx_Scoring_Val'] = approx_scoring_val


  final_season_df = pd.DataFrame(columns = ['Series', f"{p1} PTS percentile", f"{p1} TS+ percentile", f"{p1} Approx_Scoring_Val",f"{p1} MP"])

  players_df = series_data_52_22_df[(series_data_52_22_df['Player'] == p1)]
  players_df['index'] = players_df.index
  players_df = players_df.sort_values(by = ['Year', 'index'], ascending = [True, True])

  for idx, row in players_df.iterrows():
    pts = row['PTS/G']
    ts = row['TS+']
    mp = row['MP']
    pts_percentile = stats.percentileofscore(graph_data['PTS/G'], pts)
    ts_percentile = stats.percentileofscore(graph_data['TS+'], ts)
    p1_approx_scoring_val = (0.5 * pts_percentile) + (0.5 * ts_percentile)
    p1_approx_scoring_val = stats.percentileofscore(graph_data['Approx_Scoring_Val'], p1_approx_scoring_val)
    series = str(row['Year']) + " " + row['Opp']
    new_row = {'Series':series,  f"{p1} PTS percentile":pts_percentile, f"{p1} TS+ percentile": ts_percentile, f"{p1} Approx_Scoring_Val":p1_approx_scoring_val, f"{p1} MP":int(mp)}
    final_season_df = final_season_df.append(new_row, ignore_index=True)
  print(final_season_df)

In [ ]:
#@title allStretchesApproxScoringVal
def allStretchesApproxScoringVal(file, min_series_stretch):
  series_data_df = pd.read_csv(file, encoding='utf8', index_col=False)

  # create Approx_Scoring_Val for each series
  for idx, row in series_data_df.iterrows():
    pts = row['PTS/G']
    ts = row['TS+']
    mp = row['MP']
    pts_percentile = stats.percentileofscore(series_data_df['PTS/G'], pts)
    ts_percentile = stats.percentileofscore(series_data_df['TS+'], ts)
    approx_scoring_val = (0.5 * pts_percentile) + (0.5 * ts_percentile)
    series_data_df.at[idx, 'Approx_Scoring_Val'] = approx_scoring_val

  final_series_df = pd.DataFrame(columns = ['Player', 'First Series', 'Last Series', "Approx_Scoring_Val", "PTS/G", "TS+", "MP"])

  players_list = series_data_df['Player']
  players_list = players_list.drop_duplicates()

  for player in players_list:

    single_player_df = series_data_df[(series_data_df['Player'] == player)]

    new_first_series_index = 0
    num_of_series = single_player_df.shape[0]
    how_many_passes = num_of_series - min_series_stretch
    while new_first_series_index <= how_many_passes:
      tmp = single_player_df.iloc[new_first_series_index:]
      tmp = tmp.reset_index(drop=True)
      num_of_series = tmp.shape[0]
      if num_of_series < min_series_stretch:
        continue

      for series_stretch in range(min_series_stretch, num_of_series+1):

        total_mp = 0
        total_approx_scoring_val = 0
        total_pts = 0
        total_ts = 0
        mp_list = []

        idx_find_first_series = 0

        # find total minutes
        i = 0
        for mp in tmp['MP']:
          if i >= series_stretch:
            continue
          if idx_find_first_series == 0:
            first_series = str(tmp.loc[i, 'Year']) + " " + tmp.loc[i, 'Opp']
            idx_find_first_series = 1

          mp_list.append(mp)
          total_mp += mp
          last_series = str(tmp.loc[i, 'Year']) + " " + tmp.loc[i, 'Opp']
          i = i + 1

        i = 0
        for approx_scoring_val in tmp['Approx_Scoring_Val']:
          if i >= series_stretch:
            continue
          total_approx_scoring_val += approx_scoring_val * (mp_list[i] / total_mp)
          i = i + 1
        i = 0
        for pts in tmp['PTS/G']:
          if i >= series_stretch:
            continue
          total_pts += pts * (mp_list[i] / total_mp)
          i = i + 1
        i = 0
        for ts in tmp['TS+']:
          if i >= series_stretch:
            continue
          total_ts += ts * (mp_list[i] / total_mp)
          i = i + 1

        total_approx_scoring_val = '%.2f' % round(total_approx_scoring_val, 2)
        total_pts = '%.2f' % round(total_pts, 2)
        total_ts = '%.2f' % round(total_ts, 2)
        new_row = {'Player':player, 'First Series':first_series, 'Last Series':last_series, "PTS/G":total_pts, "TS+":total_ts, "Approx_Scoring_Val":total_approx_scoring_val, "MP":total_mp}
        new_df = pd.DataFrame(data=new_row, index = [0])
        final_series_df = pd.concat([final_series_df, new_df], axis=0)
        outfile = f"Stretches_Of_Series_Percentiles.csv"
        final_series_df.to_csv(outfile, index=False)
      new_first_series_index = new_first_series_index + 1

In [ ]:
allStretchesApproxScoringVal('/content/series_data_52_23.csv', 5)

In [ ]:
#@title Print Single Player Stretch
def printSinglePlayerStretch(series_stretches_file, mp_min, mp_max, unique_players, best_or_worst, first_or_last, player):

  series_data_df = pd.read_csv(series_stretches_file, encoding='utf8', index_col=False)
  series_data_df = series_data_df[(series_data_df['MP'] >= mp_min)]
  series_data_df = series_data_df[(series_data_df['MP'] <= mp_max)]

  if best_or_worst == 'best':
    series_data_df = series_data_df.sort_values('Approx_Scoring_Val', ascending=False)
  elif best_or_worst == 'worst':
    series_data_df = series_data_df.sort_values('Approx_Scoring_Val', ascending=True)
  series_data_df['Approx_Scoring_Val'] = series_data_df['Approx_Scoring_Val'].round(2)


  if unique_players == 1:
    if first_or_last == 'first':
      series_data_df = series_data_df.drop_duplicates('Player', keep='first')
    elif first_or_last == 'last':
      series_data_df = series_data_df.drop_duplicates('Player', keep='last')
  series_data_df = series_data_df.reset_index(drop=True)
  series_data_df['Historical Player Rank'] = series_data_df.index
  #print(series_data_df.shape[0])
  series_data_df = series_data_df[(series_data_df['Player'] == player)]
  series_data_df = series_data_df.drop(columns='Historical Player Rank')



  titlestring = f"\tBest Playoff Scoring Stretches (max 1 per player)\n\t\t\t\t Minimum {mp_min} MP\n"

  print(titlestring)

  print(series_data_df)

  print("\n\t'Approx_Scoring_Val' = Historical percentile of opponent adjusted PTS/G (1952-2022) * 0.5 + \n\t\t\t\tHistorical percentile of TS+ (1952-2022)* 0.5\n\n\tPTS/G adjusted based on regular season rDefRtg of playoff opponents\n\tTS+ adjusted based on opponents' TS% allowed in regular season\n\t1952-73 pace & TS% allowed both estimated")

In [ ]:
printSinglePlayerStretch('/content/Stretches_Of_Series_Percentiles.csv', 2000, 11305, 1, "best", "first", "Danny Green")

In [ ]:
#@title Print Multiple Players Stretches
def printMultiplePlayerStretches(series_stretches_file, mp_min, mp_max, list_length, unique_players, best_or_worst, first_or_last):

  series_data_df = pd.read_csv(series_stretches_file, encoding='utf8', index_col=False)
  series_data_df = series_data_df[(series_data_df['MP'] >= mp_min)]
  series_data_df = series_data_df[(series_data_df['MP'] <= mp_max)]

  if best_or_worst == 'best':
    series_data_df = series_data_df.sort_values('Approx_Scoring_Val', ascending=False)
  elif best_or_worst == 'worst':
    series_data_df = series_data_df.sort_values('Approx_Scoring_Val', ascending=True)
  series_data_df['Approx_Scoring_Val'] = series_data_df['Approx_Scoring_Val'].round(2)


  if unique_players == 1:
    if first_or_last == 'first':
      series_data_df = series_data_df.drop_duplicates('Player', keep='first')
    elif first_or_last == 'last':
      series_data_df = series_data_df.drop_duplicates('Player', keep='last')
  series_data_df = series_data_df.reset_index(drop=True)
  series_data_df['Historical Player Rank'] = series_data_df.index
  #print(series_data_df.shape[0])
  #series_data_df = series_data_df[(series_data_df['Player'] == 'James Harden') | (series_data_df['Player'] == 'Bruce Bowen') | (series_data_df['Historical Player Rank'] == 0) | (series_data_df['Historical Player Rank'] == 681)]
  #series_data_df = series_data_df[(series_data_df['Player'] == 'Manu Ginóbili') | (series_data_df['Player'] == 'Kyrie Irving') | (series_data_df['Player'] == 'Tim Duncan')]
  series_data_df = series_data_df[(series_data_df['Historical Player Rank'] <= list_length-1)]
  series_data_df = series_data_df.drop(columns='Historical Player Rank')
  #col_list = series_data_df.Player.values.tolist()
  #print(col_list)




  titlestring = f"\tBest Playoff Scoring Stretches (max 1 per player)\n\t\t\t\t Minimum {mp_min} MP\n"

  print(titlestring)

  print(series_data_df)

  series_data_df.to_csv('Top 2000 MP scoring stretches.csv',index=False)

  print("\n\t'Approx_Scoring_Val' = Historical percentile of opponent adjusted PTS/G (1952-2022) * 0.5 + \n\t\t\t\tHistorical percentile of TS+ (1952-2022)* 0.5\n\n\tPTS/G adjusted based on regular season rDefRtg of playoff opponents\n\tTS+ adjusted based on opponents' TS% allowed in regular season\n\t1952-73 pace & TS% allowed both estimated")

In [ ]:
printMultiplePlayerStretches('/content/Stretches_Of_Series_Percentiles.csv', 1500, 11305, 60, 1, "best", "first")

In [ ]:
#@title Print Percentiles of Custom Stretches
def printPercentilesofCustomStretches(series_stretches_file, player, first_opp, last_opp, mp_min, mp_max):

  series_data_df = pd.read_csv(series_stretches_file, encoding='utf8', index_col=False)
  series_data_df = series_data_df[(series_data_df['MP'] >= mp_min)]
  series_data_df = series_data_df[(series_data_df['MP'] <= mp_max)]

  player_df = series_data_df[(series_data_df['Player'] == player) & (series_data_df['First Series'] == first_opp) & (series_data_df['Last Series'] == last_opp)]
  approx_scoring_val = float(player_df['Approx_Scoring_Val'])
  print("Approx Scoring Val")
  print(approx_scoring_val)
  approx_scoring_val_perc = stats.percentileofscore(series_data_df['Approx_Scoring_Val'], approx_scoring_val)
  print("Percentile")
  print(approx_scoring_val_perc)

In [ ]:
printPercentilesofCustomStretches('/content/Stretches_Of_Series_Percentiles.csv', 'Manu Ginóbili', '2005 DEN', '2006 DAL', 1200, 12000)

In [ ]:
#@title printSeriesScoringPercentiles

# plots the approx_scoring_percentile of two players in bar graph form
series_data_52_23_df = pd.read_csv('/content/series_data_52_23.csv', encoding='utf8', index_col=False)
def printTwoSeriesScoringPercentiles(p1, p2, c1, c2, mp_min, pts_min):
  graph_data =  series_data_52_23_df.copy()
  graph_data = graph_data[(graph_data['PTS/G'] >= pts_min)]
  graph_data = graph_data[(graph_data['MP'] >= mp_min)]
  graph_data['Approx_Scoring_Val'] = 0

  for idx, row in graph_data.iterrows():
    pts = row['PTS/G']
    ts = row['TS+']
    mp = row['MP']
    pts_percentile = stats.percentileofscore(graph_data['PTS/G'], pts)
    ts_percentile = stats.percentileofscore(graph_data['TS+'], ts)
    approx_scoring_val = (0.5 * pts_percentile) + (0.5 * ts_percentile)
    graph_data.at[idx, 'Approx_Scoring_Val'] = approx_scoring_val


  final_season_df = pd.DataFrame(columns = ['Series', f"{p1} Approx_Scoring_Val", f"{p1} MP", f"{p2} Approx_Scoring_Val", f"{p2} MP"])

  players_df = series_data_52_23_df[(series_data_52_23_df['Player'] == p1) | (series_data_52_23_df['Player'] == p2)]
  players_df['index'] = players_df.index
  players_df = players_df.sort_values(by = ['Year', 'index'], ascending = [True, True])
  for idx, row in players_df.iterrows():
    round_strip = row['Round']
    round_strip = round_strip[1:]
    if row['Round'] == 'FIN':
      continue
    if round_strip == 'C1':
      round_strip = '1st'
    players_df.loc[idx, 'Round'] = round_strip

  p1_df = players_df[(players_df['Player'] == p1)]
  p2_df = players_df[(players_df['Player'] == p2)]
  p2_df = p2_df.reset_index()
  p1_df = p1_df.reset_index()

  series_done = []

  for idx, row in players_df.iterrows():

    display_series_opponent = 0
    same_team_diff_round_problem = 0

    selected_series = str(row['Player']) + str(row['Year']) + row['Opp']
    if selected_series not in series_done:
      if row['Player'] == p1:
        if row['Year'] in p2_df.values:
          other_player_year = p2_df[(p2_df['Year'] == row['Year']) & (p2_df['Round'] == row['Round'])]
          other_player_year = other_player_year.reset_index(drop=True)
          if other_player_year.empty:
            tmp_check_diff_round = p2_df[(p2_df['Year'] == row['Year']) & (p2_df['Opp'] == row['Opp'])]
            if not tmp_check_diff_round.empty:
              same_team_diff_round_problem = 1
            p2_approx_scoring_val = 0
            mp_2 = 0
            this_series = row['Player'] + str(row['Year']) + row['Opp']
            series_done.append(this_series)
            display_series_opponent = 1
          else:
            pts = other_player_year['PTS/G']
            ts = other_player_year['TS+']
            mp_2 = other_player_year['MP']
            pts_percentile = stats.percentileofscore(graph_data['PTS/G'], pts)
            ts_percentile = stats.percentileofscore(graph_data['TS+'], ts)
            p2_approx_scoring_val = (0.5 * pts_percentile) + (0.5 * ts_percentile)
            p2_approx_scoring_val = stats.percentileofscore(graph_data['Approx_Scoring_Val'], p2_approx_scoring_val)
            this_series = str(other_player_year['Player']) + str(other_player_year['Year']) + str(other_player_year['Opp'])
            series_done.append(this_series)

            if (str(other_player_year.loc[0, 'Year']) + str(other_player_year.loc[0, 'Opp'])) == (str(row['Year']) + str(row['Opp'])):
              display_series_opponent = 1
        else:
          p2_approx_scoring_val = 0
          mp_2 = 0
          display_series_opponent = 1
      if row['Player'] == p2:
        if row['Year'] in p1_df.values:
            other_player_year = p1_df[(p1_df['Year'] == row['Year']) & (p1_df['Round'] == row['Round'])]
            other_player_year = other_player_year.reset_index(drop=True)
            if other_player_year.empty:
              tmp_check_diff_round = p1_df[(p1_df['Year'] == row['Year']) & (p1_df['Opp'] == row['Opp'])]
              if not tmp_check_diff_round.empty:
                same_team_diff_round_problem = 1
              p1_approx_scoring_val = 0
              mp_1 = 0
              this_series = row['Player'] + str(row['Year']) + row['Opp']
              series_done.append(this_series)
              display_series_opponent = 1
            else:
              pts = other_player_year['PTS/G']
              ts = other_player_year['TS+']
              mp_1 = other_player_year['MP']
              pts_percentile = stats.percentileofscore(graph_data['PTS/G'], pts)
              ts_percentile = stats.percentileofscore(graph_data['TS+'], ts)
              p1_approx_scoring_val = (0.5 * pts_percentile) + (0.5 * ts_percentile)
              p1_approx_scoring_val = stats.percentileofscore(graph_data['Approx_Scoring_Val'], p1_approx_scoring_val)
              this_series = str(other_player_year['Player']) + str(other_player_year['Year']) + str(other_player_year['Opp'])
              series_done.append(this_series)

              if (str(other_player_year.loc[0, 'Year']) + str(other_player_year.loc[0, 'Opp'])) == (str(row['Year']) + str(row['Opp'])):
                display_series_opponent = 1
        else:
          p1_approx_scoring_val = 0
          mp_1 = 0
          display_series_opponent = 1

      year = row['Year']
      year = int(str(year)[2:])
      opp = row['Opp']
      year = int(year)
      if year < 10:
        year = "0" + str(year)
      else:
        year = str(year)
      series = year + " " + str(row['Round'])
      if display_series_opponent == 1:
        series = year + " " + opp
      if same_team_diff_round_problem == 1:
        series = year + " " + opp + " " + str(row['Round'])

      if row['Player'] == p1:
        pts = row['PTS/G']
        ts = row['TS+']
        mp_1 = row['MP']
        pts_percentile = stats.percentileofscore(graph_data['PTS/G'], pts)
        ts_percentile = stats.percentileofscore(graph_data['TS+'], ts)
        p1_approx_scoring_val = (0.5 * pts_percentile) + (0.5 * ts_percentile)
        p1_approx_scoring_val = stats.percentileofscore(graph_data['Approx_Scoring_Val'], p1_approx_scoring_val)
        this_series = str(row['Player']) + str(row['Year']) + row['Opp']
        series_done.append(this_series)
      else:
        pts = row['PTS/G']
        ts = row['TS+']
        mp_2 = row['MP']
        pts_percentile = stats.percentileofscore(graph_data['PTS/G'], pts)
        ts_percentile = stats.percentileofscore(graph_data['TS+'], ts)
        p2_approx_scoring_val = (0.5 * pts_percentile) + (0.5 * ts_percentile)
        p2_approx_scoring_val = stats.percentileofscore(graph_data['Approx_Scoring_Val'], p2_approx_scoring_val)
        this_series = str(row['Player']) + str(row['Year']) + row['Opp']
        series_done.append(this_series)

      new_row = {'Series':series,  f"{p1} Approx_Scoring_Val":p1_approx_scoring_val, f"{p1} MP":int(mp_1), f"{p2} Approx_Scoring_Val":p2_approx_scoring_val, f"{p2} MP":int(mp_2)}
      new_df = pd.DataFrame(new_row, index=[0])
      final_season_df = pd.concat([final_season_df, new_df], ignore_index = True)
      outfile = f"{p1}_{p2}_Series_Percentiles.csv"
      final_season_df.to_csv(outfile, index=False)

  fig, axis = plt.subplots(2)
  fig.set_figheight(40)
  fig.set_figwidth(45)

  plt.subplots_adjust(left=0.1,
                      bottom=0.1,
                      right=0.9,
                      top=0.9,
                      wspace=0.4,
                      hspace=0.4)

  axis[0].tick_params(axis='x', which='major', labelsize=14)
  axis[1].tick_params(axis='x', which='major', labelsize=14)

  perc_file_name = f"{p1}_{p2}_Series_Percentiles.csv"

  graph_data = pd.read_csv(perc_file_name, encoding='utf8', index_col=False)
  graph_data = graph_data.drop(columns=[f"{p1} MP", f"{p2} MP"])
  columns_titles = ['Series', f'{p1} Approx_Scoring_Val', f'{p2} Approx_Scoring_Val']
  graph_data = graph_data.reindex(columns=columns_titles)
  graph_data = pd.melt(graph_data, id_vars="Series", value_vars=[f'{p1} Approx_Scoring_Val', f'{p2} Approx_Scoring_Val'], value_name="Percentile", var_name ='Player Key')

  sns.barplot(data=graph_data, x='Series', y='Percentile', hue="Player Key", palette=[c1, c2], ax=axis[0])

  titlestring = f"Playoff Series Scoring [min. {pts_min} PP75; {mp_min} MP]"
  axis[0].set(title=titlestring, xlabel='Series', ylabel='Historical Percentile (1952 - 2023) *1954 not included*')

  graph_data = pd.read_csv(perc_file_name, encoding='utf8', index_col=False)
  graph_data = graph_data.drop(columns=[f'{p1} Approx_Scoring_Val', f'{p2} Approx_Scoring_Val'])
  columns_titles = ['Series', f'{p1} MP', f'{p2} MP']
  graph_data = graph_data.reindex(columns=columns_titles)
  graph_data = pd.melt(graph_data, id_vars="Series", value_vars=[f'{p1} MP', f'{p2} MP'], value_name="Percentile", var_name ='Player Key')

  sns.barplot(data=graph_data, x='Series', y='Percentile', hue="Player Key", palette=[c1, c2], ax=axis[1])

  titlestring = f"Playoff MP"
  axis[1].set(title=titlestring, xlabel='Series', ylabel='MP')


  fig.savefig(f'/content/Graph_{p1}_{p2}_Series_Percentiles', bbox_inches='tight')

  axis[0].tick_params(axis='x', which='major', labelsize=10)
  axis[1].tick_params(axis='x', which='major', labelsize=10)


def printSeriesScoringPercentiles(p1, c1, mp_min, pts_min):
  graph_data =  series_data_52_23_df.copy()
  graph_data = graph_data[(graph_data['PTS/G'] >= pts_min)]
  graph_data = graph_data[(graph_data['MP'] >= mp_min)]
  graph_data['Approx_Scoring_Val'] = 0

  for idx, row in graph_data.iterrows():
    pts = row['PTS/G']
    ts = row['TS+']
    mp = row['MP']
    pts_percentile = stats.percentileofscore(graph_data['PTS/G'], pts)
    ts_percentile = stats.percentileofscore(graph_data['TS+'], ts)
    approx_scoring_val = (0.5 * pts_percentile) + (0.5 * ts_percentile)
    graph_data.at[idx, 'Approx_Scoring_Val'] = approx_scoring_val


  final_season_df = pd.DataFrame(columns = ['Series', f"{p1} Approx_Scoring_Val", f"{p1} MP"])

  players_df = series_data_52_23_df[(series_data_52_23_df['Player'] == p1)]
  players_df['index'] = players_df.index
  players_df = players_df.sort_values(by = ['Year', 'index'], ascending = [True, True])



  for idx, row in players_df.iterrows():

      pts = row['PTS/G']
      ts = row['TS+']
      mp_1 = row['MP']
      pts_percentile = stats.percentileofscore(graph_data['PTS/G'], pts)
      ts_percentile = stats.percentileofscore(graph_data['TS+'], ts)
      p1_approx_scoring_val = (0.5 * pts_percentile) + (0.5 * ts_percentile)
      p1_approx_scoring_val = stats.percentileofscore(graph_data['Approx_Scoring_Val'], p1_approx_scoring_val)

      year = row['Year']
      year = int(str(year)[2:])
      opp = row['Opp']
      year = int(year)
      if year < 10:
        year = "0" + str(year)
      else:
        year = str(year)
      series = year + " " + opp

      new_row = {'Series':series,  f"{p1} Approx_Scoring_Val":p1_approx_scoring_val, f"{p1} MP":int(mp_1)}
      new_df = pd.DataFrame(new_row, index=[0])
      final_season_df = pd.concat([final_season_df, new_df], ignore_index = True)
      outfile = f"{p1}_Series_Percentiles.csv"
      final_season_df.to_csv(outfile, index=False)

  fig, axis = plt.subplots(2)
  fig.set_figheight(40)
  fig.set_figwidth(45)

  plt.subplots_adjust(left=0.1,
                      bottom=0.1,
                      right=0.9,
                      top=0.9,
                      wspace=0.4,
                      hspace=0.4)

  axis[0].tick_params(axis='x', which='major', labelsize=10)
  axis[1].tick_params(axis='x', which='major', labelsize=10)

  perc_file_name = f"{p1}_Series_Percentiles.csv"

  graph_data = pd.read_csv(perc_file_name, encoding='utf8', index_col=False)
  graph_data = graph_data.drop(columns=[f"{p1} MP"])
  columns_titles = ['Series', f'{p1} Approx_Scoring_Val']
  graph_data = graph_data.reindex(columns=columns_titles)
  graph_data = pd.melt(graph_data, id_vars="Series", value_vars=[f'{p1} Approx_Scoring_Val'], value_name="Percentile", var_name ='Player Key')

  sns.barplot(data=graph_data, x='Series', y='Percentile', hue="Player Key", palette=[c1], ax=axis[0])

  titlestring = f"Playoff Series Scoring [min. {pts_min} PP75; {mp_min} MP]"
  axis[0].set(title=titlestring, xlabel='Series', ylabel='Historical Percentile (1952 - 2022) *1954 not included*')

  graph_data = pd.read_csv(perc_file_name, encoding='utf8', index_col=False)
  graph_data = graph_data.drop(columns=[f'{p1} Approx_Scoring_Val'])
  columns_titles = ['Series', f'{p1} MP']
  graph_data = graph_data.reindex(columns=columns_titles)
  graph_data = pd.melt(graph_data, id_vars="Series", value_vars=[f'{p1} MP'], value_name="Percentile", var_name ='Player Key')

  sns.barplot(data=graph_data, x='Series', y='Percentile', hue="Player Key", palette=[c1], ax=axis[1])

  titlestring = f"Playoff MP"
  axis[1].set(title=titlestring, xlabel='Series', ylabel='MP')


  fig.savefig(f'/content/Graph_{p1}_Series_Percentiles', bbox_inches='tight')

In [ ]:
printTwoSeriesScoringPercentiles("Magic Johnson", "Manu Ginóbili", "g", "r", 120, 5)

In [ ]:
printSeriesScoringPercentiles("Tony Parker", "k", 120, 5)

In [ ]:
#@title printOrderedSeriesScoringPercentiles

# plots the approx_scoring_percentile of two players in bar graph form
series_data_52_22_df = pd.read_csv('/content/series_data_52_23.csv', encoding='utf8', index_col=False)
def printOrderedSeriesScoringPercentiles(p1, p2, c1, c2, mp_min, pts_min):
  graph_data =  series_data_52_22_df.copy()
  graph_data = graph_data[(graph_data['PTS/G'] >= pts_min)]
  graph_data = graph_data[(graph_data['MP'] >= mp_min)]
  graph_data['Approx_Scoring_Val'] = 0

  for idx, row in graph_data.iterrows():
    pts = row['PTS/G']
    ts = row['TS+']
    mp = row['MP']
    pts_percentile = stats.percentileofscore(graph_data['PTS/G'], pts)
    ts_percentile = stats.percentileofscore(graph_data['TS+'], ts)
    approx_scoring_val = (0.5 * pts_percentile) + (0.5 * ts_percentile)
    graph_data.at[idx, 'Approx_Scoring_Val'] = approx_scoring_val


  final_season_df = pd.DataFrame(columns = ['Series', 'Player', "Approx_Scoring_Val", "MP"])

  players_df = series_data_52_22_df[(series_data_52_22_df['Player'] == p1) | (series_data_52_22_df['Player'] == p2)]

  for idx, row in players_df.iterrows():
    year = row['Year']
    year = int(str(year)[2:])
    opp = row['Opp']
    year = int(year)
    if year < 10:
      year = "0" + str(year)
    else:
      year = str(year)
    series = year + " " + opp
    pts = row['PTS/G']
    ts = row['TS+']
    mp = row['MP']
    pts_percentile = stats.percentileofscore(graph_data['PTS/G'], pts)
    ts_percentile = stats.percentileofscore(graph_data['TS+'], ts)
    approx_scoring_val = (0.5 * pts_percentile) + (0.5 * ts_percentile)
    approx_scoring_val = stats.percentileofscore(graph_data['Approx_Scoring_Val'], approx_scoring_val)
    new_row = {'Series':series,  'Player': row['Player'], "Approx_Scoring_Val": approx_scoring_val, "MP":int(mp)}
    final_season_df = final_season_df.append(new_row, ignore_index=True)
  outfile = f"{p1}_{p2}_Ordered_Series_Percentiles.csv"
  final_season_df.to_csv(outfile, index=False)

  fig, axis = plt.subplots(2)
  fig.set_figheight(40)
  fig.set_figwidth(45)

  plt.subplots_adjust(left=0.1,
                      bottom=0.1,
                      right=0.9,
                      top=0.9,
                      wspace=0.4,
                      hspace=0.4)

  axis[0].tick_params(axis='x', which='major', labelsize=10)
  axis[1].tick_params(axis='x', which='major', labelsize=10)

  perc_file_name = f"{p1}_{p2}_Ordered_Series_Percentiles.csv"

  graph_data = pd.read_csv(perc_file_name, encoding='utf8', index_col=False)
  graph_data
  first_cond = (graph_data['Player'] == p1)
  graph_data['color'] = np.where(first_cond==True, c1, "#00B2A9")
  graph_data['color'] = np.where((graph_data.Player == p2), c2, graph_data.color)
  graph_data = graph_data.sort_values('Approx_Scoring_Val', ascending = False)
  graph_data = graph_data.reset_index()
  sns.set(font_scale=2.5)

  sns.barplot(data=graph_data, x=graph_data.index, y='Approx_Scoring_Val', palette=graph_data.color, ax=axis[0])

  titlestring = f"Playoff Series Scoring [min. {pts_min} PP75; {mp_min} MP]"
  axis[0].set(title=titlestring, xlabel='Series', ylabel='Historical Percentile (1952 - 2022) *1954 not included*')
  axis[0].set(xticklabels=[])
  axis[0].set(xlabel=None)
  axis[0].tick_params(bottom=False)  # remove the ticks

  legend_elements = [Line2D([0], [0], marker='$\u25A0$', color='#EAEAF2', label=p2, markerfacecolor=c2,  markersize=20),
                    Line2D([0], [0], marker='$\u25A0$', color='#EAEAF2', label=p1, markerfacecolor=c1,  markersize=20)]
  axis[0].legend(handles=legend_elements, loc='upper right')


  sns.barplot(data=graph_data, x=graph_data.index, y='MP', palette=graph_data.color, ax=axis[1])
  titlestring = f"Playoff MP"
  axis[1].set(title=titlestring, xlabel='Series', ylabel='MP')
  axis[1].set(xticklabels=[])
  axis[1].set(xlabel=None)
  axis[1].tick_params(bottom=False)  # remove the ticks

  axis[1].legend(handles=legend_elements, loc='upper right')


  fig.savefig(f'/content/Graph_{p1}_{p2}_Ordered_Series_Percentiles', bbox_inches='tight')



  # Downward Single Plot
  #perc_file_name = f"{p1}_{p2}_Ordered_Series_Percentiles.csv"
  #graph_data = pd.read_csv(perc_file_name, encoding='utf8', index_col=False)

  #first_cond = (graph_data['Player'] == p1)
  #graph_data['color'] = np.where(first_cond==True, c1, "#00B2A9")
  #graph_data['color'] = np.where((graph_data.Player == p2), c2, graph_data.color)
  #graph_data['MP'] = graph_data['MP'] * -1
  #graph_data = graph_data.sort_values('MP', ascending = False)
  #graph_data = graph_data.reset_index()

  #fig = plt.figure(figsize=(40,45), tight_layout=True)


  #ax = sns.barplot(data=graph_data, x=graph_data.index, y='Approx_Scoring_Val', palette=graph_data.color)
  #ax = sns.barplot(data=graph_data, x=graph_data.index, y='MP', palette=["k", "k"])
  #ax.tick_params(axis='x', which='major', labelsize=10)

  #titlestring = f"Playoff Series Scoring [min. {pts_min} PP75; {mp_min} MP]"
  #ax.set(title=titlestring, xlabel='Series', ylabel='Historical Percentile (1952 - 2022) *1954 not included*')


  #fig.savefig(f'/content/Graph_{p1}_{p2}_Ordered_Series_Percentiles', bbox_inches='tight')

In [ ]:
#@title printOrderedSeriesScoringPercentiles (MPG)

# plots the approx_scoring_percentile of two players in bar graph form
def printMPGOrderedSeriesScoringPercentiles(p1, p2, c1, c2, mp_min, pts_min, option):
  graph_data = pd.read_csv('/content/series_data_52_22.csv', encoding='utf8', index_col=False)
  from scipy import stats
  graph_data = graph_data[(graph_data['PTS/G'] >= pts_min)]
  graph_data = graph_data[(graph_data['MP'] >= mp_min)]
  graph_data['Approx_Scoring_Val'] = 0

  for idx, row in graph_data.iterrows():
    pts = row['PTS/G']
    ts = row['TS+']
    mp = row['MP']
    pts_percentile = stats.percentileofscore(graph_data['PTS/G'], pts)
    ts_percentile = stats.percentileofscore(graph_data['TS+'], ts)
    approx_scoring_val = (0.5 * pts_percentile) + (0.5 * ts_percentile)
    graph_data.at[idx, 'Approx_Scoring_Val'] = approx_scoring_val


  final_season_df = pd.DataFrame(columns = ['Series', 'Player', "Approx_Scoring_Val", "MP", "MPG", "index"])


  players_df = pd.read_csv('/content/MPG_data.csv', encoding='utf8', index_col=False)
  players_df['index'] = players_df.index
  players_df = players_df.sort_values(by = ['Year', 'index'], ascending = [True, True])

  for idx, row in players_df.iterrows():
    index = row['index']
    year = row['Year']
    year = int(str(year)[2:])
    opp = row['Opp']
    year = int(year)
    if year < 10:
      year = "0" + str(year)
    else:
      year = str(year)
    series = year + " " + opp
    pts = row['PTS/G']
    ts = row['TS+']
    mp = row['MP']
    mpg = row['MPG']
    pts_percentile = stats.percentileofscore(graph_data['PTS/G'], pts)
    ts_percentile = stats.percentileofscore(graph_data['TS+'], ts)
    approx_scoring_val = (0.5 * pts_percentile) + (0.5 * ts_percentile)
    approx_scoring_val = stats.percentileofscore(graph_data['Approx_Scoring_Val'], approx_scoring_val)
    new_row = {'Series':series,  'Player': row['Player'], "Approx_Scoring_Val": approx_scoring_val, "MP":int(mp), "MPG":float(mpg), "index": index}
    final_season_df = final_season_df.append(new_row, ignore_index=True)
  outfile = f"{p1}_{p2}_Ordered_Series_Percentiles.csv"
  final_season_df.to_csv(outfile, index=False)

  if option == 3:
    fig, axis = plt.subplots(3)
    fig.set_figheight(40)
    fig.set_figwidth(45)

    plt.subplots_adjust(left=0.1,
                        bottom=0.1,
                        right=0.9,
                        top=0.9,
                        wspace=0.4,
                        hspace=0.4)

    axis[0].tick_params(axis='x', which='major', labelsize=10)
    axis[1].tick_params(axis='x', which='major', labelsize=10)

    perc_file_name = f"{p1}_{p2}_Ordered_Series_Percentiles.csv"

    graph_data = pd.read_csv(perc_file_name, encoding='utf8', index_col=False)
    graph_data
    first_cond = (graph_data['Player'] == p1)
    graph_data['color'] = np.where(first_cond==True, c1, "#00B2A9")
    graph_data['color'] = np.where((graph_data.Player == p2), c2, graph_data.color)
    graph_data = graph_data.sort_values('Approx_Scoring_Val', ascending = False)
    graph_data = graph_data.reset_index()
    sns.set(font_scale=2.5)

    sns.barplot(data=graph_data, x=graph_data.index, y='Approx_Scoring_Val', palette=graph_data.color, ax=axis[0])

    titlestring = f"Playoff Series Scoring [min. {pts_min} PP75; {mp_min} MP]"
    axis[0].set(title=titlestring, xlabel='Series', ylabel='Historical Percentile (1952 - 2022) *1954 not included*')
    axis[0].set(xticklabels=[])
    axis[0].set(xlabel=None)
    axis[0].tick_params(bottom=False)  # remove the ticks

    legend_elements = [Line2D([0], [0], marker='$\u25A0$', color='#EAEAF2', label=p2, markerfacecolor=c2,  markersize=20),
                      Line2D([0], [0], marker='$\u25A0$', color='#EAEAF2', label=p1, markerfacecolor=c1,  markersize=20)]
    axis[0].legend(handles=legend_elements, loc='upper right')

    sns.barplot(data=graph_data, x=graph_data.index, y='MPG', palette=graph_data.color, ax=axis[1])
    titlestring = f"Playoff MPG"
    axis[1].set(title=titlestring, xlabel='Series', ylabel='MPG')
    axis[1].set(xticklabels=[])
    axis[1].set(xlabel=None)
    axis[1].tick_params(bottom=False)  # remove the ticks

    axis[1].legend(handles=legend_elements, loc='upper right')


    fig.savefig(f'/content/Graph_{p1}_{p2}_Ordered_Series_Percentiles', bbox_inches='tight')


    sns.barplot(data=graph_data, x=graph_data.index, y='MP', palette=graph_data.color, ax=axis[2])
    titlestring = f"Playoff MP"
    axis[2].set(title=titlestring, xlabel='Series', ylabel='MP')
    axis[2].set(xticklabels=[])
    axis[2].set(xlabel=None)
    axis[2].tick_params(bottom=False)  # remove the ticks

    axis[2].legend(handles=legend_elements, loc='upper right')


    fig.savefig(f'/content/Graph_{p1}_{p2}_Ordered_Series_Percentiles', bbox_inches='tight')
  else:
    perc_file_name = f"{p1}_{p2}_Ordered_Series_Percentiles.csv"
    graph_data = pd.read_csv(perc_file_name, encoding='utf8', index_col=False)

    first_cond = (graph_data['Player'] == p1)
    graph_data['color'] = np.where(first_cond==True, c1, "#00B2A9")
    graph_data['color'] = np.where((graph_data.Player == p2), c2, graph_data.color)
    graph_data['MPG'] = graph_data['MPG'] * -1
    graph_data = graph_data.sort_values('Approx_Scoring_Val', ascending = False)
    graph_data = graph_data.reset_index()

    fig = plt.figure(figsize=(20,8), tight_layout=True)
    sns.set(font_scale=1.0)

    ax = sns.barplot(data=graph_data, x=graph_data.index, y='Approx_Scoring_Val', palette=graph_data.color)
    ax = sns.barplot(data=graph_data, x=graph_data.index, y='MPG', palette=["k", "k"])
    ax.tick_params(axis='x', which='major', labelsize=10)

    titlestring = f"Playoff Series Scoring [min. {pts_min} PP75; {mp_min} MP]"
    ax.set(title=titlestring, xlabel='Series', ylabel='Historical Percentile (1952 - 2022) *1954 not included*')
    ax.set(xticklabels=[])
    ax.set(xlabel=None)
    ax.tick_params(bottom=False)  # remove the ticks
    legend_elements = [Line2D([0], [0], marker='$\u25A0$', color='#EAEAF2', label=p2, markerfacecolor=c2,  markersize=8),
                      Line2D([0], [0], marker='$\u25A0$', color='#EAEAF2', label=p1, markerfacecolor=c1,  markersize=8)]
    ax.legend(handles=legend_elements, loc='upper right')
    ticks =  ax.get_yticks()
    ax.set_yticklabels([int(abs(tick)) for tick in ticks])


    fig.savefig(f'/content/Graph_{p1}_{p2}_Ordered_Series_Percentiles', bbox_inches='tight')